# Using tracker.py

In [14]:
%matplotlib inline
# Imports
from matplotlib import pyplot as plt
plt.ion()
import numpy as np
import pandas as pd
import os
import tracker
import importlib
importlib.reload(tracker)


<module 'tracker' from '/Users/jordan/Dropbox/Ircam/code/tracker.py'>

In [15]:
## RhythmData

# RhythmData objects have three definng properties:
# - beat_onset (list of times in seconds)
# - bar (list of bar numbers counting upward from 0)
# - beat (index within bar of each beat, startin from 1)\
# beat_onset is mandatory, but by default (infer=True) it will assume 4/4 time and infer bar/beat as needed.

rd = tracker.RhythmData(np.arange(0,10,0.4))

# Basic information, like the .period() (median of beat interval) and .first_db() are automatically summarized:
rd

<Rhythm period:0.4000000000000001 nbeats:25 first:0.0>

In [16]:
# .df() returns onset/bar/beat as a pandas dataframe.
rd.df().head(8)

,bar,beat,onset
0,0,1,0.0
1,0,2,0.4
2,0,3,0.8
3,0,4,1.2
4,1,1,1.6
5,1,2,2.0
6,1,3,2.4
7,1,4,2.8


In [17]:
# Some ways to quickly create variations on a given rhythm:

# (1) .shift_beats(X) shifts the downbeat position earlier by X beats:
# The beat_onset times will be unchanged.

print(rd.df().head())

print(rd.shift_beats(1).df().head())

print("Onsets same: " + str(np.all(rd.beat_onset==rd.shift_beats(1).beat_onset)))
print("Forward == backward: " + str(np.all(rd.shift_beats(1).downbeats()==rd.shift_beats(-3).downbeats())))

   bar  beat  onset
0    0     1    0.0
1    0     2    0.4
2    0     3    0.8
3    0     4    1.2
4    1     1    1.6
   bar  beat  onset
0    0     2    0.0
1    0     3    0.4
2    0     4    0.8
3    1     1    1.2
4    1     2    1.6
Onsets same: True
Forward == backward: True


In [18]:
# (2) .inject_beats(submeter)
# AKA beats -> downbeats
# Using existing beats as downbeats, inject new sub-beats in between them with fixed spacing.
print(rd.inject_beats(3).df().head(10))

# (3) .superject_beats(supermeter, phase_offset=0)
# AKA downbeats -> beats
# Count the downbeats as beats and interpret a hypermeter with a particular phase.
# Phase offset is added to the index of the first beat.
print(pd.concat((rd.superject_beats(3,0).df(), rd.superject_beats(3,1).df()), axis=1))


   bar  beat     onset
0    1     1  0.000000
1    1     2  0.133333
2    1     3  0.266667
3    2     1  0.400000
4    2     2  0.533333
5    2     3  0.666667
6    3     1  0.800000
7    3     2  0.933333
8    3     3  1.066667
9    4     1  1.200000
   bar  beat  onset  bar  beat  onset
0    0     1    0.0    0     2    0.0
1    0     2    1.6    0     3    1.6
2    0     3    3.2    1     1    3.2
3    1     1    4.8    1     2    4.8
4    1     2    6.4    1     3    6.4
5    1     3    8.0    2     1    8.0
6    2     1    9.6    2     2    9.6


In [19]:
## Beat

# The "Beat" object is just a wrapper object that lets you look at and manipulate RhythmData objects.
beat = tracker.Beat()

# Lots of directories are hard-coded into Beat, such as:
print(beat.database_path)
print(beat.sv_dir)
print(beat.audio_orig_dir)
print(beat.audio_solo_dir)

/Users/jordan/Documents/data/WeimarJazzDatabase/wjazzd.db
/Users/jordan/Documents/data/WeimarJazzDatabase/annotations/SV/
/Users/jordan/Documents/data/WeimarJazzDatabase/audio/wav_orig/
/Users/jordan/Documents/data/WeimarJazzDatabase/audio/wav_solo/


In [20]:
# The object, when initailised, connects to wjazzd.db, and gets all its ground truth info there. Example:
beat.load_true_beats_and_downbeats(melid=33)
beat.beats['true'].df().head(10)

,bar,beat,onset
0,1,1,2.960544
1,1,2,3.462132
2,1,3,3.958912
3,1,4,4.447120
4,2,1,4.918027
5,2,2,5.416916
6,2,3,5.895850
7,2,4,6.356825
8,3,1,6.825941
9,3,2,7.286848


In [27]:
# The .set_index(ind) method is a quick way to set:
# .load_audio(ind)
# .load_true_beats_and_downbeats(ind)
# .ind = ind
# beat.track_info.loc[30:60,:]

beat.set_index(41)
# print(beat.signal_mono.shape)
# print(beat.S_mono.shape)

Loading audio...


FileNotFoundError: [Errno 2] No such file or directory: '/Users/jordan/Documents/data/WeimarJazzDatabase/audio/wav_solo/BranfordMarsalis_TheNearnessOfYou_Solo.wav'

In [28]:
# A beat-tracking run by hand:
melid=41
beat.ind=melid

beat.load_audio(melid)
# audio stored in beat.signal_mono
# complex and real spectra stored in beat.S_mono and beat.V_mono, respectively

beat.estimate_beats(extractor_type='qm')
beat.estimate_beats(extractor_type='essentia')
beat.estimate_beats(extractor_type='madmom') # This one takes the longest.
# beat-tracked output stored in beat.beats[extractor_type]
print(beat.beats['qm'])

# NB: essentia does not estimate downbeats, but we infer them as being in 4/4 starting on the first beat.
# print beat.beats['essentia'].df().head(10)


Loading audio...


FileNotFoundError: [Errno 2] No such file or directory: '/Users/jordan/Documents/data/WeimarJazzDatabase/audio/wav_solo/BranfordMarsalis_TheNearnessOfYou_Solo.wav'

In [10]:
# Evaluation by hand:

# beat level:
beat.load_true_beats_and_downbeats(melid)
scores = tracker.get_scores(ref_beats = beat.beats['true'].beat_onset,
                            est_beats = beat.beats['qm'].beat_onset)

# get_scores returns: [f_measure, goto, p_score, information_gain] + list(cemgil) + list(continuity)
# (there are 2 values in cemgil and 4 in continuity)

# downbeat level:
scores_db = tracker.get_scores(ref_beats = beat.beats['true'].downbeats(),
                               est_beats = beat.beats['qm'].downbeats())

print(scores)
print(scores_db)

[0.5397350993377483, 0.0, 0.4071246819338422, 0.22529928757726322, 0.3723895453298955, 0.5512576361389141, 0.0, 0.0, 0.5876777251184834, 0.8151658767772512]
[0.5165562913907283, 0.0, 0.494949494949495, 0.5619439555478365, 0.3644257307003742, 0.5448345082748169, 0.0, 0.0, 0.9423076923076923, 0.9423076923076923]


In [11]:
# Quick evaluation of phase and period:

period_err, phase_err = tracker.get_phase_and_period_error(beat.beats['madmom'], beat.beats['true'], 'beats')

a,b,c,d = period_err
# a = est_period
# b = true_period
# c = est_period/true_period
# d = true_period/est_period

e,f,g,h = phase_err
# e = median(est_to_true)
# f = median(true_to_est)
# g = median(est_to_true)/true_period
# h = median(true_to_est)/est_period

# I decided that c and g above are the most intuitive metrics.
# c : near 1 is better (ranges from 0 to inf)
# g : near 0 is better (ranges from 0 to 0.5)

print(c, g)

KeyError: 'madmom'

In [12]:
# Quickly evaluate the full corpus:
melids = np.unique(beat.solo_info.melid)
methods = ['qm','essentia','madmom']
score_list = []
for melid in melids: #(This will take about 20 seconds to process all 450 solo files)
    beat.load_true_beats_and_downbeats(melid)
    beat.ind = melid
    beat.load_estimates()
    errs = [tracker.get_phase_and_period_error(beat.beats[method], beat.beats['true'], 'downbeats') for method in methods]
    est_to_true_period = np.array(zip(*errs)[0])[:,2]
    est_to_true_phase = np.array(zip(*errs)[1])[:,2]
    score_list += [[est_to_true_period, est_to_true_phase]]

scores = np.array(score_list)
# Look at scores(i,j,k) for:
#     i = melid
#     j = index into [period, phase]
#     k = index into [qm, essentia, or madmom]]

In [14]:
plt.hist(scores[:,0,2], 20)
# plt.subplot(1,2,2)
# plt.hist(scores[:,1,2], 20)


(array([ 23., 268.,   2.,   2., 152.,   1.,   1.,   0.,   0.,   2.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   5.]),
 array([0.33139357, 0.71968802, 1.10798248, 1.49627693, 1.88457139,
        2.27286585, 2.6611603 , 3.04945476, 3.43774921, 3.82604367,
        4.21433813, 4.60263258, 4.99092704, 5.37922149, 5.76751595,
        6.15581041, 6.54410486, 6.93239932, 7.32069377, 7.70898823,
        8.09728269]),
 <a list of 20 Patch objects>)

AttributeError: 'module' object has no attribute 'to_rgba'